In [3]:

import numpy as np
import torch
import scipy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# !pip install datasets
# !pip install evaluate
from datasets import load_dataset
import evaluate
from torch.utils.data import DataLoader

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
# Load the dataset
dataset = load_dataset("ugursa/Yahoo-Finance-News-Sentences")


In [6]:
# Split the dataset into training and evaluation sets using the datasets library
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

In [7]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [9]:
# Tokenize the dataset
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/20025 [00:00<?, ? examples/s]

Map:   0%|          | 0/5007 [00:00<?, ? examples/s]

In [10]:
# Load the metric
metric = evaluate.load("accuracy")

In [11]:
# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [12]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
model.to(device)  # Move model to GPU if available

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    save_total_limit=3,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

In [ ]:
# Evaluate the model
eval_dataloader = DataLoader(tokenized_eval_dataset, batch_size=16)

In [ ]:
!pip install torch

import torch

In [ ]:
# def evaluate_model(model, dataloader):
#     model.eval()
#     all_predictions = []
#     all_labels = []
#     all_confidences = []

#     with torch.no_grad():
#         for batch in dataloader:
#             inputs = {key: value.to(device) for key, value in batch.items() if key != "labels"}
#             labels = batch["labels"].to(device)

#             outputs = model(**inputs)
#             logits = outputs.logits
#             predictions = torch.argmax(logits, dim=-1)

#             # Calculate confidence levels (softmax)
#             softmax = torch.nn.functional.softmax(logits, dim=-1)
#             confidences, _ = torch.max(softmax, dim=-1)

#             all_predictions.extend(predictions.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             all_confidences.extend(confidences.cpu().numpy())

#     # Calculate accuracy
#     accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
#     # Calculate average confidence
#     avg_confidence = np.mean(all_confidences)

#     return accuracy, avg_confidence




# def evaluate_model(model, dataloader):
#     model.eval()
#     all_predictions = []
#     all_labels = []
#     all_confidences = []

#     with torch.no_grad():
#         for batch in dataloader:
#             # Convert list values to tensors

#             labels = batch["labels"].to(device)

#             outputs = model(**inputs)
#             logits = outputs.logits
#             predictions = torch.argmax(logits, dim=-1)

#             # Calculate confidence levels (softmax)
#             softmax = torch.nn.functional.softmax(logits, dim=-1)
#             confidences, _ = torch.max(softmax, dim=-1)

#             all_predictions.extend(predictions.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             all_confidences.extend(confidences.cpu().numpy())

#     # Calculate accuracy
#     accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
#     # Calculate average confidence
#     avg_confidence = np.mean(all_confidences)

#     return accuracy, avg_confidence

# accuracy, avg_confidence = evaluate_model(model, eval_dataloader)
# print(f"Accuracy: {accuracy}")
# print(f"Average Confidence: {avg_confidence}")


import torch
import numpy as np

def evaluate_model(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_labels = []
    all_confidences = []

    with torch.no_grad():
        for batch in dataloader:
            # Debug: Print batch keys
            print(f"Batch keys: {batch.keys()}")

            if "label" not in batch:
                raise KeyError("Batch does not contain 'label' key.")

            labels = batch["label"].to(device)

            # Move tensor-like inputs to device
            inputs = {}
            for k, v in batch.items():
                if k in ["input_ids", "attention_mask", "token_type_ids"]:
                    if isinstance(v, torch.Tensor):
                        inputs[k] = v.to(device)
                    else:
                        try:
                            # Convert list or numpy array to tensor
                            v = np.array(v)  # Convert list to numpy array first
                            inputs[k] = torch.tensor(v, dtype=torch.long).to(device)
                        except Exception as e:
                            print(f"Error converting {k} to tensor: {e}")
                            raise

            # Debug: Print inputs keys and shapes
            print(f"Inputs keys: {inputs.keys()}")
            for key, value in inputs.items():
                print(f"Shape of {key}: {value.shape}")

            # Ensure required keys are in the inputs
            if "input_ids" not in inputs:
                raise ValueError("Batch does not contain 'input_ids' key which is required for the model.")

            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Debug: Print shapes of logits and predictions
            print(f"Shape of logits: {logits.shape}")
            print(f"Shape of predictions: {predictions.shape}")
            print(f"Shape of labels: {labels.shape}")

            # Calculate confidence levels (softmax)
            softmax = torch.nn.functional.softmax(logits, dim=-1)
            confidences, _ = torch.max(softmax, dim=-1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_confidences.extend(confidences.cpu().numpy())

    # Debug: Print lengths of all_predictions and all_labels
    print(f"Length of all_predictions: {len(all_predictions)}")
    print(f"Length of all_labels: {len(all_labels)}")

    # Calculate accuracy
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    # Calculate average confidence
    avg_confidence = np.mean(all_confidences)

    print(f"Accuracy: {accuracy}")
    print(f"Average Confidence: {avg_confidence}")

    return accuracy, avg_confidence

# Example usage:
# Assuming `model` and `eval_dataloader` are defined and `device` is set

accuracy, avg_confidence = evaluate_model(model, eval_dataloader, torch.device("cuda"))
print(f"Accuracy: {accuracy}")
print(f"Average Confidence: {avg_confidence}")












In [ ]:
# Example usage:
# Assuming `model` and `eval_dataloader` are defined and `device` is set


accuracy, avg_confidence = evaluate_model(model, eval_dataloader, torch.device("cuda"))
print(f"Accuracy: {accuracy}")
print(f"Average Confidence: {avg_confidence}")